ref: https://musicinformationretrieval.com/ipython_audio.html

In [88]:
%load_ext autoreload
%autoreload 2
%pylab inline
import pandas as pd
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


In [89]:
import librosa
import librosa.display
import IPython.display as ipd

In [90]:
def strip_audio(x, frame_length=1024, hop_length=256, rms_ths=0.2):
    # compute energy
    energy = numpy.array([
        sum(abs(x[i:i+frame_length]**2))
        for i in range(0, len(x), hop_length)
    ])
    rmse = librosa.feature.rmse(x, frame_length=frame_length, hop_length=hop_length)[0]
    rms_ratio = rmse/rmse.max()
    active_frames = np.nonzero(rms_ratio > rms_ths)[0]
    if len(active_frames) == 0:
        return x
    # strip continous active part
    s_sample = librosa.frames_to_samples(active_frames[0], hop_length=hop_length)[0]
    e_sample = librosa.frames_to_samples(active_frames[-1], hop_length=hop_length)[0]  
#     frames = range(len(energy))
#     t = librosa.frames_to_time(frames, sr=sr, hop_length=hop_length)
#     librosa.display.waveplot(x, sr=sr, alpha=0.4)
#     plt.plot(t, energy/energy.max(), 'r--')             # normalized for visualization
#     plt.plot(t[:len(rmse)], rmse/rmse.max(), color='g') # normalized for visualization
#     plt.legend(('Energy', 'RMSE'))

    return x[s_sample:e_sample]

In [4]:
def _timeshift_audio(self, data):
    shift = (16000 * self.timeshift_ms) // 1000
    shift = random.randint(-shift, shift)
    a = -min(0, shift)
    b = max(0, shift)
    data = np.pad(data, (a, b), "constant")
    return data[:len(data) - a] if a else data[b:]

### Speech Commands

In [92]:
dataframe_dir = '/home/muncok/DL/dataset/SV_sets/dataframes/'
data_dir = '/home/muncok/DL/dataset/SV_sets/speech_commands/'
command_df = pd.read_pickle(os.path.join(dataframe_dir,'Command_Dataframe.pkl'))

In [103]:
import soundfile as sf

for i, row in command_df.iterrows():
    file_path = os.path.join(data_dir, row.sent, row.file)
    save_path = 'vad/speech_commands/{}'.format(row.sent)
    if not os.path.exists(save_path): 
        os.makedirs(save_path)
    x, sr = sf.read(file_path)
    striped_x = strip_audio(x, rms_ths=0.2)
    sf.write(os.path.join(save_path, row.file), striped_x, sr)

/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  
/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in greater
  if __name__ == '__main__':
